# Parse {year,type,county} subtotals from NJSP PDFs

NJSP pages for each year (example: [2023](https://www.nj.gov/njsp/info/fatalacc/2023-stats.shtml)) include links to two PDFs:
- Year to Date Comparative ([example](https://www.nj.gov/njsp/info/fatalacc/pdf/swfcs2_23.pdf))
- Victim Classification by County ([example](https://www.nj.gov/njsp/info/fatalacc/pdf/ptccr_23.pdf))

These provide some info that's not available elsewhere:
- {year, victim type, county} subtotals
- {year, victim type, age range} subtotals

These are available for all but 4 years since 2008 (the earliest NJSP data): 2008, 2009, 2017, and 2018. Later in this notebook we recover {year,type} subtotals (sans "county" facet) for those years. [read_gpt_csvs.ipynb](../annual-reports/year-type-county/read_gpt_csvs.ipynb) also recovers {year,type,county} stats for those years.

NJSP's per-crash records (`data/FAUQStats*.xml` files) only include "victim type" info since 2020, so this notebook backfills {year,type} subtotals for 2008-2019. It produces `data/year_types.csv`, which `njsp-plots.ipynb` uses as part of a daily Github Action.

In [1]:
from utz import *
from tabula import read_pdf
from njsp.paths import CRASHES_PQT, ANNUAL_SUMMARIES, ANNUAL_SUMMARIES_YT_CSV, ANNUAL_SUMMARIES_YTC_CSV, MISSING_YTC, annual_ytc_relpath
from njsp.utils import cur_year, parse_years

Papermill parameters:

In [2]:
years = None

In [3]:
# Parameters
years = "2023"


In [4]:
START = 2010
all_years = list(range(START, cur_year()))
if years is None:
    load_existing_years = False
    years = all_years
else:
    # This notebook updates year-type-county.csv with all years' data; if executing
    # only on certain years, load in existing CSV and patch in new data.
    load_existing_years = True
    years = parse_years(years, start=START)
years

[2023]

Tabula helpers

In [5]:
def load_rects(tpl_name):
    tpl_path = f'{ANNUAL_SUMMARIES}/{tpl_name}.json'
    with open(tpl_path, 'r') as f:
        tpl = json.load(f)
    return tpl

def load_pdf_tbl(rect, pdf_path):
    [tbl] = read_pdf(pdf_path, area=[ rect[k] for k in [ 'y1', 'x1', 'y2', 'x2', ] ], pages='all',)
    return tbl

## Load "Victim Classification by County" data

In [6]:
[ptccr_rect] = load_rects('ptccr_23.tabula-template')
def load_ptccr(year):
    pdf_path = annual_ytc_relpath(year)
    tbl = load_pdf_tbl(ptccr_rect, pdf_path)
    tbl['year'] = year
    tbl = tbl.set_index('County')
    return tbl

In [7]:
cur_year = now().year
cur_year

2024

In [8]:
%%time
missing_years = [ 2008, 2009, 2017, 2018 ]
summaries = pd.concat([
    load_ptccr(year)
    for year in years
    if year not in missing_years
])
summaries.columns = summaries.columns.str.lower()
summaries = summaries.rename(columns={
    'pedalcyclist': 'cyclist',
})
summaries.index.name = 'county'
summaries.reset_index().set_index(['year', 'county'])
summaries

Failed to import jpype dependencies. Fallback to subprocess.


No module named 'jpype'


Got stderr: Dec 31, 2024 3:27:27 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Dec 31, 2024 3:27:27 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Dec 31, 2024 3:27:27 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Dec 31, 2024 3:27:27 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>
Dec 31, 2024 3:27:27 PM org.apache.pdfbox.pdmodel.font.PDType1Font <init>



CPU times: user 9.71 ms, sys: 704 μs, total: 10.4 ms
Wall time: 1.23 s


,driver,passenger,cyclist,pedestrian,fatalities,crashes,year
county,,,,,,,
Atlantic,18,7,2,13,40,37,2023
Bergen,20,4,1,13,38,36,2023
Burlington,26,3,2,5,36,34,2023
Camden,18,7,5,11,41,40,2023
Cape May,4,1,0,2,7,7,2023
Cumberland,13,5,0,4,22,20,2023
Essex,23,5,2,23,53,49,2023
Gloucester,21,7,1,5,34,32,2023
Hudson,10,3,3,9,25,23,2023


Verify "fatalities" is the sum of the 4 types:

In [9]:
type_cols = ['driver', 'passenger', 'cyclist', 'pedestrian']
assert (summaries[type_cols].sum(1) == summaries.fatalities).all()

Verify "Total" (per year) rows:

In [10]:
assert (summaries.drop(index='Total').groupby('year').sum() == summaries.set_index('year', append=True).loc['Total']).all().all()

Load YTC counts for missing years (2008, 2009, 2017, 2018) (computed by [read_gpt_csvs.ipynb](../annual-reports/year-type-county/read_gpt_csvs.ipynb)):

In [11]:
missing_ytc = read_csv(MISSING_YTC).set_index(['year', 'county']).astype(int)
missing_ytc

crashes  driver  passenger  pedestrian  cyclist
year county                                                     
2008 Atlantic         30      17          8           6        0
     Bergen           22      10          5           7        1
     Burlington       45      23          6          12        4
     Camden           42      25          4          15        0
     Cape May         11       8          3           0        0
     Cumberland       23      15          5           2        1
     Essex            43      23         11          14        1
     Gloucester       29      14         13           5        1
     Hudson           24      13          4           6        4
     Hunterdon         9       6          2           3        0
     Mercer           20      11          6           4        1
     Middlesex        42      28          9           8        1
     Monmouth         45      29          9           7        2
     Morris           25      15          2           6        3
     Ocean            37      20          8          10        0
     Passaic          25       9          5          11        0
     Salem            14      13          3           0        0
     Somerset         15       8          2           5        0
     Sussex           14      12          2           0        0
     Union            35      17          4          16        1
     Warren            5       4          1           1        0
2009 Atlantic         41      22          8          12        1
     Bergen           25      14          0          12        0
     Burlington       43      27         14           7        0
     Camden           37      24          7           9        2
     Cape May          8       7          0           1        0
     Cumberland       27      21          5           1        1
     Essex            40      14          7          21        0
     Gloucester       22      14          4           3        1
     Hudson           22       7          3          12        0
     Hunterdon         8       5          1           2        0
     Mercer           21      11          4           7        0
     Middlesex        44      19         10          18        1
     Monmouth         34      18          2          11        3
     Morris           18      12          3           3        0
     Ocean            57      32         12          15        4
     Passaic          23      16          5           5        0
     Salem            13      11          1           1        1
     Somerset         13       9          3           1        0
     Sussex            7       5          2           0        0
     Union            38      21          6          14        0
     Warren            9       6          1           2        0
2017 Atlantic         34      21          5          10        0
     Bergen           25      13          5           8        1
     Burlington       46      31          5          12        0
     Camden           40      22          6          15        1
     Cape May         15       9          5           2        0
     Cumberland       24      16          4           5        1
     Essex            39      14          3          22        1
     Gloucester       43      25          9           9        1
     Hudson           23       6          2          15        3
     Hunterdon         8       7          0           1        0
     Mercer           26      13          2          11        0
     Middlesex        44      26          7          13        2
     Monmouth         40      25          6          11        1
     Morris           26      16          5           7        1
     Ocean            52      28          8          13        4
     Passaic          18       9          4           5        0
     Salem            15      16          1           0        0
     Somerset         22      13          2         

In [12]:
idx_cols = ['year', 'county']
ytc = pd.concat([
    summaries.drop(index='Total').drop(columns='fatalities').reset_index().set_index(idx_cols),
    missing_ytc,
])
if load_existing_years:
    existing = read_csv(ANNUAL_SUMMARIES_YTC_CSV).set_index(idx_cols)
    ytc = (
        pd.concat([ ytc, existing ])
        .reset_index()
        .drop_duplicates(subset=idx_cols)
        .set_index(idx_cols)
    )
ytc = ytc.sort_index()
ytc

driver  passenger  cyclist  pedestrian  crashes
year county                                                     
2008 Atlantic        17          8        0           6       30
     Bergen          10          5        1           7       22
     Burlington      23          6        4          12       45
     Camden          25          4        0          15       42
     Cape May         8          3        0           0       11
...                 ...        ...      ...         ...      ...
2023 Salem            8          2        0           2       11
     Somerset        13          4        0           6       21
     Sussex           6          2        0           1        6
     Union           14          6        2          15       35
     Warren           7          1        0           3       11

[336 rows x 5 columns]

In [13]:
ytc.reset_index(level=1).county.value_counts()

county
Atlantic      16
Middlesex     16
Union         16
Sussex        16
Somerset      16
Salem         16
Passaic       16
Ocean         16
Morris        16
Monmouth      16
Mercer        16
Bergen        16
Hunterdon     16
Hudson        16
Gloucester    16
Essex         16
Cumberland    16
Cape May      16
Camden        16
Burlington    16
Warren        16
Name: count, dtype: int64

In [14]:
ytc.to_csv(ANNUAL_SUMMARIES_YTC_CSV)

In [15]:
yt = ytc.reset_index(level=1, drop=True).groupby(lambda x:x).sum()
yt

,driver,passenger,cyclist,pedestrian,crashes
year,,,,,
2008,320,112,20,138,555
2009,315,98,14,157,550
2010,303,99,13,141,530
2011,362,105,17,143,586
2012,309,103,14,163,553
2013,304,92,14,132,508
2014,295,80,11,170,523
2015,276,96,17,173,522
2016,330,89,17,166,570


In [16]:
yt.to_csv(ANNUAL_SUMMARIES_YT_CSV)

## Compare to per-crash records
NJSP's per-crash datasets include victim-type info for crashes since 2020.

This allows for a sanity-check of the {year,type,county} totals reported in the "Victim Classification by County" PDFs (for 2020-2023):

In [17]:
sp = read_parquet(CRASHES_PQT)
sp

,cc,mc,dt,tk,ti,dk,ok,pk,bk,location,street,highway
id,,,,,,,,,,,,
1703,1,2,2008-01-01 00:35:00-05:00,1,1,<NA>,<NA>,<NA>,<NA>,State/Interstate Authority 446 S MP 1,None,446
1681,9,10,2008-01-01 04:11:00-05:00,1,<NA>,<NA>,<NA>,<NA>,<NA>,Bergenline Ave S MP 0 at 6th St,Bergenline Ave,None
1659,4,15,2008-01-01 06:46:00-05:00,1,1,<NA>,<NA>,<NA>,<NA>,State Highway 42 N MP 8.2,None,42
1661,20,4,2008-01-01 12:29:00-05:00,1,1,<NA>,<NA>,<NA>,<NA>,County 624 W MP 2.2 at Ikea Dr,None,624
1811,7,16,2008-01-01 18:53:00-05:00,1,<NA>,<NA>,<NA>,<NA>,<NA>,County 648 E MP .87 at Franklin Ave,None,648
...,...,...,...,...,...,...,...,...,...,...,...,...
13664,16,2,2024-12-24 22:47:00-05:00,1,<NA>,1,0,0,0,Ackerman Ave,Ackerman Ave,None
13667,15,7,2024-12-27 05:04:00-05:00,1,<NA>,1,0,0,0,State Highway 70,None,70
13679,15,7,2024-12-28 08:28:00-05:00,1,<NA>,1,0,0,0,70,None,70


In [18]:
from njdot.data import cc2cn

renames = {
    'tk': 'fatalities',
    'dk': 'driver',
    'ok': 'passenger',
    'pk': 'pedestrian',
    'bk': 'cyclist',
}
cols = list(renames.keys())
y = sp.dt.dt.year.rename('year')
c = sp.cc.map(cc2cn).rename('county')
gb = sp.groupby([y, c])
agg = gb[cols].sum(numeric_only=True).astype(int)
agg['crashes'] = gb.size()
agg = agg.rename(columns=renames)
agg20 = agg.reset_index(level=1)
agg20 = agg20[agg20.index >= 2020]
assert (agg20[type_cols].sum(1) == agg20.fatalities).all()
agg = agg.drop(columns='fatalities')
agg

driver  passenger  pedestrian  cyclist  crashes
year county                                                     
2008 Atlantic         0          0           0        0       30
     Bergen           0          0           0        0       22
     Burlington       0          0           0        0       45
     Camden           0          0           0        0       42
     Cape May         0          0           0        0       11
...                 ...        ...         ...      ...      ...
2024 Salem            9          3           1        3       14
     Somerset        12          0           8        0       19
     Sussex           8          2           2        1       13
     Union           18          6          13        1       34
     Warren          12          1           2        1       15

[357 rows x 5 columns]

### Combine aggregate stats from per-crash records vs. stats from PDFs

In [19]:
m = agg20.reset_index()
m = m.merge(summaries.reset_index(), how='left', on=['year', 'county'], suffixes=['_sp', '_stats']).dropna()
m = m.set_index(['year', 'county'])
m = m[m.columns.sort_values()].astype(int)
m.columns = pd.MultiIndex.from_tuples([ tuple(reversed(col.split('_'))) for col in m.columns ])
m = m[m.columns.sort_values()].astype(int)
m

sp                                                 \
                crashes cyclist driver fatalities passenger pedestrian   
year county                                                              
2023 Atlantic        37       2     18         40         7         13   
     Bergen          36       1     20         38         4         13   
     Burlington      34       2     26         36         3          5   
     Camden          40       5     18         41         7         11   
     Cape May         7       0      4          7         1          2   
     Cumberland      20       0     13         22         5          4   
     Essex           49       2     23         53         5         23   
     Gloucester      32       1     21         34         7          5   
     Hudson          23       3     10         25         3          9   
     Hunterdon        4       1      3          4         0          0   
     Mercer          28       1     14         28         3         10   
     Middlesex       57       2     28         59         9         20   
     Monmouth        37       4     17         37         7          9   
     Morris          22       0     14         22         4          4   
     Ocean           40       1     27         43         7          8   
     Passaic         24       0     15         25         2          8   
     Salem           11       0      8         12         2          2   
     Somerset        21       0     13         23         4          6   
     Sussex           6       0      6          9         2          1   
     Union           35       2     14         37         6         15   
     Warren          11       0      7         11         1          3   

                  stats                                                 
                crashes cyclist driver fatalities passenger pedestrian  
year county                                                             
2023 Atlantic        37       2     18         40         7         13  
     Bergen          36       1     20         38         4         13  
     Burlington      34       2     26         36         3          5  
     Camden          40       5     18         41         7         11  
     Cape May         7       0      4          7         1          2  
     Cumberland      20       0     13         22         5          4  
     Essex           49       2     23         53         5         23  
     Gloucester      32       1     21         34         7          5  
     Hudson          23       3     10         25         3          9  
     Hunterdon        4       1      3          4         0          0  
     Mercer          28       1     14         28         3         10  
     Middlesex       57       2     28         59         9         20  
     Monmouth        37       4     17         37         7          9  
     Morris          22       0     14         22         4          4  
     Ocean           40       1     27         43         7          8  
     Passaic         24       0     15         25         2          8  
     Salem           11       0      8         12         2          2  
     Somerset        21       0     13         23         4          6  
     Sussex           6       0      6          9         2          1  
     Union           35       2     14         37         6         15  
     Warren          11       0      7         11         1          3

In [20]:
row_diffs = (m['sp'] == m['stats']).all(1)
row_diffs = row_diffs[~row_diffs]
row_diffs

Series([], dtype: bool)

In [21]:
assert row_diffs.all(), row_diffs